In [ ]:
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import torch
import sys

sys.path.append('..')
# from huggingface_hub import login

# login()  # login with your User Access Token, found at https://huggingface.co/settings/tokens


In [ ]:
torch.tensor([0.485, 0.456, 0.406]).reshape(1, -1, 1, 1)

In [ ]:

# pretrained=True needed to load UNI2-h weights (and download weights for the first time)
timm_kwargs = {
            'img_size': 224, 
            'patch_size': 14, 
            'depth': 24,
            'num_heads': 24,
            'init_values': 1e-5, 
            'embed_dim': 1536,
            'mlp_ratio': 2.66667*2,
            'num_classes': 0, 
            'no_embed_class': True,
            'mlp_layer': timm.layers.SwiGLUPacked, 
            'act_layer': torch.nn.SiLU, 
            'reg_tokens': 8, 
            'dynamic_img_size': True
        }
model = timm.create_model("hf-hub:MahmoodLab/UNI2-h", pretrained=True, **timm_kwargs)
transform = create_transform(**resolve_data_config(model.pretrained_cfg, model=model))
model.eval()


In [ ]:
from PIL import Image
image = Image.open("/home/valentin/workspaces/benchmark-vfm-ss/data/ANORAK/image/train001_Da382.png")
# image = transform(image).unsqueeze(dim=0) # Image (torch.Tensor) with shape [1, 3, 224, 224] following image resizing and normalization (ImageNet parameters)
image = torch.rand(1, 3, 224, 448)  
with torch.inference_mode():
    feature_emb = model.forward_features(image)

In [ ]:
feature_emb.shape  # torch.Size([1, 1536, 16, 8])

In [ ]:
model.patch_embed.grid_size

In [ ]:
model = model.train()

In [ ]:
model.pos_drop

In [ ]:
model.patch_embed(image).shape

In [ ]:
model.cls_token.shape

In [ ]:
model.num_prefix_tokens

In [ ]:
model.pos_embed.shape

In [ ]:
model.embed_dim

In [ ]:
model.patch_embed


In [ ]:
from models.histo.encoder import ViTEncoderPyramidHooks

In [ ]:
pyramid_model = ViTEncoderPyramidHooks(
    vit=model,
    embed_dim=None,
    pyramid_channels=None,
    extract_layers=[6, 12, 18, 24],
)

In [ ]:
with torch.inference_mode():
    out = pyramid_model(image)

In [ ]:
out["s32"].shape

In [ ]:
image.shape